<a href="https://colab.research.google.com/github/fenzhantw/DLforeveryone/blob/main/DEEPLEARNING_DNN_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#colab tensorflow 2.0.0 세팅

In [2]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.8.0


In [9]:
pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3 MB 41 kB/s 
     |████████████████████████████████| 50 kB 5.3 MB/s 
     |████████████████████████████████| 449 kB 68.0 MB/s 
     |████████████████████████████████| 3.8 MB 41.4 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=187dc6077932bb6a99dfed9aa7f98486a9b3f29f7d71a30f7660098742d946e4
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Success

In [106]:
!pip3 check_util

ERROR: unknown command "check_util" - maybe you meant "check"


#모듈

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

In [90]:
def normalize(train_data,test_data):
  train_data = train_data.astype(np.float32)/255.0
  test_data = test_data.astype(np.float32)/255.0
  return train_data,test_data


def load_mnist():
  (train_data,train_labels),(test_data,test_labels) = fashion_mnist.load_data()
  train_data,test_data = normalize(train_data,test_data)
  train_labels = to_categorical(train_labels,10)
  test_labels = to_categorical(test_labels,10)
  
  return train_data,train_labels,test_data,test_labels

def flatten():
  return tf.keras.layers.Flatten()

def dense(channel,weight_init):
  return tf.keras.layers.Dense(units=channel,use_bias=True,kernel_initializer=weight_init)

def relu():
  return tf.keras.layers.Activation(tf.keras.activations.relu)

def loss_fn(model,images,labels):
  #model에 image를 넣고, logit을 구함
  logits = model(images,training=True)
  #model에서 구한 softmax(logit)로 예측한 값과 lables의 차이를 loss를 구함
  loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
  return loss

def accuracy_fn(model,images,labels):
  logits = model(images,training=False)
  prediction = tf.equal(tf.argmax(logits,-1),tf.argmax(labels,-1))
  accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
  return accuracy

def grad(model,images,labels):
  with tf.GradientTape() as tape:
    loss = loss_fn(model,images,labels)
  return tape.gradient(loss,model.variables)

def batch_norm():
  return tf.keras.layers.BatchNormalization()

In [91]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128
epochs = 10
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [92]:
class create_model(tf.keras.Model):
  def __init__(self,label_dim):
    super(create_model,self).__init__()

    weight_init = tf.keras.initializers.RandomNormal()
    self.model = tf.keras.Sequential()
    self.model.add(flatten()) # [N,28,28,1] -> [N,784]
    self.model.add(relu())

    def call(self,x,training=None,mask=None):
      x=self.model(x)

      return x

In [93]:
""" Model """
network = create_model_class(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [95]:
for epoch in range(0,1):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=idx)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=idx)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=idx)

                print(
                    "Epoch: [%2d] [%5d/%5d], train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, train_loss, train_accuracy,
                       test_accuracy))            

Epoch: [ 0] [    0/  468], train_loss: 0.37182546, train_accuracy: 0.8516, test_Accuracy: 0.8434
Epoch: [ 0] [    1/  468], train_loss: 0.30052453, train_accuracy: 0.8672, test_Accuracy: 0.8452
Epoch: [ 0] [    2/  468], train_loss: 0.35496068, train_accuracy: 0.8594, test_Accuracy: 0.8475
Epoch: [ 0] [    3/  468], train_loss: 0.38144958, train_accuracy: 0.8672, test_Accuracy: 0.8520
Epoch: [ 0] [    4/  468], train_loss: 0.27718461, train_accuracy: 0.8828, test_Accuracy: 0.8536
Epoch: [ 0] [    5/  468], train_loss: 0.37871978, train_accuracy: 0.8516, test_Accuracy: 0.8524
Epoch: [ 0] [    6/  468], train_loss: 0.23018263, train_accuracy: 0.9141, test_Accuracy: 0.8499
Epoch: [ 0] [    7/  468], train_loss: 0.26464406, train_accuracy: 0.8906, test_Accuracy: 0.8513
Epoch: [ 0] [    8/  468], train_loss: 0.32026157, train_accuracy: 0.8750, test_Accuracy: 0.8507
Epoch: [ 0] [    9/  468], train_loss: 0.21428783, train_accuracy: 0.9219, test_Accuracy: 0.8475
Epoch: [ 0] [   10/  468], tra

In [107]:
# test phase      
for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)
        print("test_Accuracy: %.4f" % (test_accuracy))

test_Accuracy: 0.8500
